# Data Evaluation <a id='data-evaluation'>

In [1]:
# standard libraries
import pandas as pd
import numpy as np
import os
from IPython.display import Image

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tabulate import tabulate

# data pre-processing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# prediction models
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

#from kneed import KneeLocator
from scipy import stats

# import warnings filter
'''import warnings
warnings.filterwarnings('ignore')
from warnings import simplefilter 
simplefilter(action='ignore', category=FutureWarning)'''

"import warnings\nwarnings.filterwarnings('ignore')\nfrom warnings import simplefilter \nsimplefilter(action='ignore', category=FutureWarning)"

# Loading Data <a id='loading-data'>

In [48]:
url = 'https://raw.githubusercontent.com/olmosjorge28/QTW-SPRING-2022/main/dataset_diabetes/IDs_mapping.csv'
admission_type_mapping = pd.read_csv(url, nrows=8, index_col=0)
admission_type_mapping

,description
admission_type_id,
1,Emergency
2,Urgent
3,Elective
4,Newborn
5,Not Available
6,NaN
7,Trauma Center
8,Not Mapped


Can Potentially Deduce values from medical specialty

In [54]:
discharge_disposition_mapping = pd.read_csv(url, nrows=30,skiprows=10, index_col=0 )
discharge_disposition_mapping 

,description
discharge_disposition_id,
1,Discharged to home
2,Discharged/transferred to another short term h...
3,Discharged/transferred to SNF
4,Discharged/transferred to ICF
5,Discharged/transferred to another type of inpa...
6,Discharged/transferred to home with home healt...
7,Left AMA
8,Discharged/transferred to home under care of H...
9,Admitted as an inpatient to this hospital


In [98]:
admission_source_mapping = pd.read_csv(url,skiprows=42, index_col=0 )
admission_source_mapping

,description
admission_source_id,
1,Physician Referral
2,Clinic Referral
3,HMO Referral
4,Transfer from a hospital
5,Transfer from a Skilled Nursing Facility (SNF)
6,Transfer from another health care facility
7,Emergency Room
8,Court/Law Enforcement
9,Not Available


# Referral,Transfer,Emergency, Normal Delivery, Other Delivery, Other/NA

In [404]:
url = 'https://raw.githubusercontent.com/olmosjorge28/QTW-SPRING-2022/main/dataset_diabetes/diabetic_data.csv'
df = pd.read_csv(url,na_values='?')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [405]:
df.columns.values.tolist()

['encounter_id',
 'patient_nbr',
 'race',
 'gender',
 'age',
 'weight',
 'admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'time_in_hospital',
 'payer_code',
 'medical_specialty',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'diag_1',
 'diag_2',
 'diag_3',
 'number_diagnoses',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed',
 'readmitted']

## Types of Data

### Qualitative

#### Nominal:

#### Ordinal:

### Quantitative

#### Discrete

#### Continous

In [78]:
medical_specialty_groupby =df.groupby(['medical_specialty','admission_type_id']).size( )

In [82]:
admission_type_groupby = df.groupby(['admission_type_id','medical_specialty']).size( )

In [80]:
medical_specialty_groupby.to_csv('medical_specialty_groupby.csv')

In [83]:
admission_type_groupby.to_csv('admission_type_groupby.csv')

## Remap Id Mappings

In [383]:
def remapIds(*new_mappings: tuple) -> dict:
    mapping_dict = dict()
    for mapping in new_mappings:
        mapping_dict[mapping[1]] =  mapping[0].map(mapping[2])
    return mapping_dict
        

In [394]:
def refactorMappingIds(inputDf: pd.DataFrame) -> pd.DataFrame:
    disharge_disposition_new_mapping = {
        1: "discharged",
        2: "transfer",
        3: "transfer",
        4: "transfer",
        5: "transfer",
        6: "transfer",
        7: "ama",
        8: "tansfer",
        9: "admitted",
        10: "transfer",
        11: "expired",
        12: "admitted",
        13: "hospice",
        14: "hospice",
        15: "transfer",
        16: "transfer",
        17: "transfer",
        18: float("NaN"),
        19: "expired",
        20: "expired",
        21: "expired",
        22: "transfer",
        23: "transfer",
        24: "transfer",
        25: float("NaN"),
        26: float("NaN"),
        27: "transfer",
        28: "transfer",
        29: "transfer",
    }


    admission_sourcing_new_mapping = {
        1: "referral",
        2: "referral",
        3: "referral",
        4: "transfer",
        5: "transfer",
        6: "transfer",
        7: "emergency",
        8: "law-enforcement",
        9: float("NaN"),
        10: "transfer",
        11: "normal-delivery",
        12: "other-delivery",
        13: "other-delivery",
        14: "other-delivery",
        15: float("NaN"),
        17: float("NaN"),
        18: "transfer",
        19: "transfer",
        20: float("NaN"),
        21: float("NaN"),
        22: "transfer",
        23: "normal-delivery",
        24: "normal-delivery",
        25: "transfer",
        26: "transfer"
    }
    df = inputDf.copy()
    mapping_tuples = [
        (df['discharge_disposition_id'],'disharge_disposition_new_mapping', disharge_disposition_new_mapping),
        (df['admission_source_id'],'admission_source_new_mapping', admission_sourcing_new_mapping)
    ]
    for newMappingKey in remapIds(*mapping_tuples):
        df[newMappingKey] = remappings[newMappingKey]
    return df

In [395]:
df = refactorMappingIds(df)

## Recategorizing Diag_1, Diag_2, Diag_3

In [306]:
def getDiagCategory(input: float) -> str:
    val: str
    if input < 1:
        val = float("NaN")
    elif input < 140:
        val = 'A'
    elif input < 240:
        val = 'B'
    elif input < 280:
        val = 'C'
    elif input < 290:
        val = 'D'
    elif input < 320:
        val = 'E'
    elif input < 390:
        val = 'F'
    elif input < 460:
        val = 'G'
    elif input < 520:
        val = 'H'
    elif input < 580:
        val = 'I'
    elif input < 630:
        val = 'J'
    elif input < 680:
        val = 'K'
    elif input < 710:
        val = 'L'
    elif input < 740:
        val = 'M'
    elif input < 760:
        val = 'N'
    elif input < 780:
        val = 'O'
    elif input < 800:
        val = 'P'
    elif input < 1000:
        val = 'Q'
    elif input < 2000:
        val = 'R'
    elif input < 3000:
        val = 'S'
    else:
        val = 'Z'
    return val
    

def categorizeDiag(diag: pd.Series) -> pd.Series:
    df = diag.copy()
    df.fillna(0,inplace=True)
    df.mask(df.str.startswith('V', na=False),1000, inplace=True)
    df.mask(df.str.startswith('E', na=False),2000, inplace=True)
    df = pd.to_numeric(df)
    df = df.map(getDiagCategory)
    return df
    

In [399]:
def recategorizeDiags(inputDf: pd.DataFrame) -> pd.DataFrame:
    df = inputDf.copy()
    df['diag_1_categorized'] = categorizeDiag(df['diag_1'])
    df['diag_2_categorized'] = categorizeDiag(df['diag_2'])
    df['diag_3_categorized'] = categorizeDiag(df['diag_3'])
    return df
    

In [400]:
df = recategorizeDiags(df)

### Inserting New Variables Addition variables

In [402]:
def addDiag(*diags):
    df = None
    for diag in diags:
        if (df is None):
            df = diag.notna().astype(int)
        else:
            df = df + diag.notna().astype(int)
    return df

In [403]:
def recategorizeData(inputDf: pd.DataFrame) -> pd.DataFrame:
    df = inputDf.copy()
    df = refactorMappingIds(df)
    df = recategorizeDiags(df)
    df['total_diag'] = addDiag(df['diag_1'],df['diag_2'],df['diag_3'])
    return df
    

In [406]:
df2 = recategorizeData(df)

In [407]:
df2.columns.values.tolist()

['encounter_id',
 'patient_nbr',
 'race',
 'gender',
 'age',
 'weight',
 'admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'time_in_hospital',
 'payer_code',
 'medical_specialty',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'diag_1',
 'diag_2',
 'diag_3',
 'number_diagnoses',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed',
 'readmitted',
 'disharge_disposition_new_mapping',
 'admission_source_new_mapping',
 'diag_1_categorized',
 'diag_2_categorized',
 'diag_3_categori

## Missing Data

In [312]:
df.isna().sum()

encounter_id                            0
patient_nbr                             0
race                                 2273
gender                                  0
age                                     0
weight                              98569
admission_type_id                       0
discharge_disposition_id                0
admission_source_id                     0
time_in_hospital                        0
payer_code                          40256
medical_specialty                   49949
num_lab_procedures                      0
num_procedures                          0
num_medications                         0
number_outpatient                       0
number_emergency                        0
number_inpatient                        0
diag_1                                 21
diag_2                                358
diag_3                               1423
number_diagnoses                        0
max_glu_serum                           0
A1Cresult                         

In [408]:
df2.isna().sum()

encounter_id                            0
patient_nbr                             0
race                                 2273
gender                                  0
age                                     0
weight                              98569
admission_type_id                       0
discharge_disposition_id                0
admission_source_id                     0
time_in_hospital                        0
payer_code                          40256
medical_specialty                   49949
num_lab_procedures                      0
num_procedures                          0
num_medications                         0
number_outpatient                       0
number_emergency                        0
number_inpatient                        0
diag_1                                 21
diag_2                                358
diag_3                               1423
number_diagnoses                        0
max_glu_serum                           0
A1Cresult                         

In [409]:
df2.dtypes

encounter_id                         int64
patient_nbr                          int64
race                                object
gender                              object
age                                 object
weight                              object
admission_type_id                    int64
discharge_disposition_id             int64
admission_source_id                  int64
time_in_hospital                     int64
payer_code                          object
medical_specialty                   object
num_lab_procedures                   int64
num_procedures                       int64
num_medications                      int64
number_outpatient                    int64
number_emergency                     int64
number_inpatient                     int64
diag_1                              object
diag_2                              object
diag_3                              object
number_diagnoses                     int64
max_glu_serum                       object
A1Cresult  

## Variables Not Needed

patient_nbr <br>
encounter_id <br>
examide <br>
citoglipton <br>
weight <br>

In [410]:
df2 = df2.drop(['patient_nbr','encounter_id', 'examide', 'weight'], 1)

## Variables Removed Because of Recategorization

In [ ]:
discharge_disposition_id
admission_source_id
diag_1
diag_2
diag_3

In [411]:
df2 = df2.drop(['discharge_disposition_id','admission_source_id','diag_1',
                'diag_2','diag_3'], 1)

In [412]:
df2.columns.values.tolist()

['race',
 'gender',
 'age',
 'admission_type_id',
 'time_in_hospital',
 'payer_code',
 'medical_specialty',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed',
 'readmitted',
 'disharge_disposition_new_mapping',
 'admission_source_new_mapping',
 'diag_1_categorized',
 'diag_2_categorized',
 'diag_3_categorized',
 'total_diag']

In [413]:
df2.to_csv('diabetic_data_recategorized.csv')

KNN, Total Elimnation, Mode/Mean/Median

https://www.verywellhealth.com/finding-icd-codes-2615311

https://www.aapc.com/codes/icd9-codes-range/